### Lab 5
- Juan Pablo Solis
- Brandon Reyes
- Carlos Valladares

### Task 1.1 Discretizacion Imagen

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy.ndimage import label
from abc import ABC, abstractmethod
from collections import deque
import heapq

In [2]:
def process_maze(image_path):
    image = Image.open(image_path).convert("RGB")
    image_array = np.array(image)
    height, width, _ = image_array.shape

    black_walls = (image_array[:, :, 0] < 50) & (image_array[:, :, 1] < 50) & (image_array[:, :, 2] < 50)
    red_start = (image_array[:, :, 0] > 200) & (image_array[:, :, 1] < 100) & (image_array[:, :, 2] < 100)
    green_goal = (image_array[:, :, 1] > 200) & (image_array[:, :, 0] < 100) & (image_array[:, :, 2] < 100)

    red_positions = np.where(red_start)
    if len(red_positions[0]) > 0:
        cell_size = max(red_positions[0].max() - red_positions[0].min(), red_positions[1].max() - red_positions[1].min())
    else:
        cell_size = 10

    grid_height = height // cell_size
    grid_width = width // cell_size
    grid = np.ones((grid_height, grid_width, 3), dtype=np.uint8) * 255  # Fondo blanco

    labeled_goals, num_goals = label(green_goal)
    goals = []
    for goal_id in range(1, num_goals + 1):
        goal_pixels = np.where(labeled_goals == goal_id)
        center_y = int(goal_pixels[0].mean()) // cell_size
        center_x = int(goal_pixels[1].mean()) // cell_size
        grid[center_y, center_x] = [0, 255, 0]  # Verde
        goals.append((center_y, center_x))

    start = None
    if len(red_positions[0]) > 0:
        red_cell_x = red_positions[1].min() // cell_size
        red_cell_y = red_positions[0].min() // cell_size
        grid[red_cell_y, red_cell_x] = [255, 0, 0]  # Rojo
        start = (red_cell_y, red_cell_x)

    for i in range(grid_height):
        for j in range(grid_width):
            block = image_array[i * cell_size:(i + 1) * cell_size, j * cell_size:(j + 1) * cell_size]
            avg_color = block.mean(axis=(0, 1))
            if avg_color[0] < 50 and avg_color[1] < 50 and avg_color[2] < 50:
                grid[i, j] = [0, 0, 0]  # Pared negra

    return grid, start, goals

Task 1.2 

In [3]:
# ===================== Task 1.2: Definición del Framework =====================
class MazeProblem(ABC):
    def __init__(self, grid):
        self.grid = grid

    @abstractmethod
    def actions(self, state):
        pass

    @abstractmethod
    def step_cost(self, state, action, next_state):
        pass

    @abstractmethod
    def is_goal(self, state):
        pass

class ConcreteMazeProblem(MazeProblem):
    def __init__(self, grid, start, goals):
        super().__init__(grid)
        self.start = start
        self.goals = goals

    def actions(self, state):
        moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        valid_moves = []
        for move in moves:
            new_state = (state[0] + move[0], state[1] + move[1])
            if 0 <= new_state[0] < self.grid.shape[0] and 0 <= new_state[1] < self.grid.shape[1] and not np.array_equal(self.grid[new_state[0], new_state[1]], [0, 0, 0]):
                valid_moves.append(move)
        return valid_moves

    def step_cost(self, state, action, next_state):
        return 1

    def is_goal(self, state):
        return state in self.goals
